# Setup

In [ ]:
# https://www.geeksforgeeks.org/how-to-run-cuda-c-c-on-jupyter-notebook-in-google-colaboratory/

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

# Kode CUDA

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include <cuda_runtime.h>
#include <cuComplex.h>
#define MAX_N 512
#define BLOCK_SIZE 16

struct Matrix {
    int size;
    double mat[MAX_N][MAX_N];
};

struct FreqMatrix {
    int size;
    cuDoubleComplex mat[MAX_N][MAX_N];
};

__device__ cuDoubleComplex cuCexp(cuDoubleComplex x)
{
    double factor = exp(x.x);
    return make_cuDoubleComplex(factor * cos(x.y), factor * sin(x.y));
}

__global__ void dft_kernel(struct Matrix *mat, struct FreqMatrix *freq_domain)
{
  // Implement shared memory
    __shared__ double shared_mat[BLOCK_SIZE][BLOCK_SIZE];

    int k = blockIdx.x;
    int l = threadIdx.x;

    cuDoubleComplex element = make_cuDoubleComplex(0.0, 0.0);

    for (int i = 0; i < mat->size; i += BLOCK_SIZE) {
        for (int j = 0; j < mat->size; j += BLOCK_SIZE) {
            // Load a block of input matrix into shared memory
            shared_mat[l][k] = mat->mat[i + l][j + k];

            __syncthreads();

            for (int m = 0; m < BLOCK_SIZE; m++) {
                for (int n = 0; n < BLOCK_SIZE; n++) {
                    cuDoubleComplex arg      = make_cuDoubleComplex((i + m) * k / (double) mat->size + (j + n) * l / (double) mat->size, 0.0);
                    cuDoubleComplex exponent = cuCexp(make_cuDoubleComplex(0.0, -2.0 * M_PI * arg.x));
                    cuDoubleComplex value    = make_cuDoubleComplex(shared_mat[m][k], 0.0);
                    element = cuCadd(element, cuCmul(value, exponent));
                }
            }

            __syncthreads();
        }
    }

    element = cuCdiv(element, make_cuDoubleComplex(mat->size*mat->size, 0.0));
    freq_domain->mat[k][l] = element;
}

void readMatrix(struct Matrix *m)
{
    scanf("%d", &(m->size));
    for (int i = 0; i < m->size; i++)
        for (int j = 0; j < m->size; j++)
            scanf("%lf", &(m->mat[i][j]));
}

int main(void)
{
    struct Matrix source;
    struct FreqMatrix freq_domain;
    readMatrix(&source);
    freq_domain.size = source.size;

    // Allocate device memory
    struct Matrix *d_source;
    cudaMalloc(&d_source, sizeof(struct Matrix));
    cudaMemcpy(d_source, &source, sizeof(struct Matrix), cudaMemcpyHostToDevice);

    struct FreqMatrix *d_freq_domain;
    cudaMalloc(&d_freq_domain, sizeof(struct FreqMatrix));
    cudaMemcpy(d_freq_domain, &freq_domain, sizeof(struct FreqMatrix), cudaMemcpyHostToDevice);

    // Launch kernel
    dft_kernel<<<source.size, source.size>>>(d_source, d_freq_domain);

    // Copy results back to host
    cudaMemcpy(&freq_domain, d_freq_domain, sizeof(struct FreqMatrix), cudaMemcpyDeviceToHost);

    // Free device memory
    cudaFree(d_source);
    cudaFree(d_freq_domain);

    cuDoubleComplex sum = make_cuDoubleComplex(0.0, 0.0);
    for (int k = 0; k < source.size; k++) {
        for (int l = 0; l < source.size; l++) {
            cuDoubleComplex el = freq_domain.mat[k][l];
            printf("(%lf, %lf) ", el.x, el.y);
            sum = cuCadd(sum, el);
        }
        printf("\n");
    }
    return 0;
};

Overwriting cuda.cu


# Write CUDA

In [ ]:
!nvcc cuda.cu -o cuda

# Test CUDA with Testcase

In [ ]:
!time ./cuda < 512.txt > output.txt


real	0m2.798s
user	0m1.109s
sys	0m0.743s
